# **Tarefa 1 – Web Scraping em Ambiente Controlado (50%)**
### **Disciplina**: Coleta, Preparação e Análise de Dados - Prof. Lucas Pessutto

#### **Integrantes do grupo**: Ana Cristina Schmidt e Jeniffer Borges

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re 
import time
from datetime import datetime
import csv
import os

In [2]:
os.makedirs('html_dos_sites', exist_ok=True)
os.makedirs('dados_em_csv', exist_ok=True)

url_base = 'http://127.0.0.1:8000/places/default/index/'

url_pais = 'http://127.0.0.1:8000'

In [3]:
def extrair_dados_pais(pais_link):
    response = requests.get(pais_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        pais = soup.find('tr', id='places_country__row')
        pais_nome = pais.find('td', class_='w2p_fw').text.strip() if pais else "Nao encontrado"

        arq_html(pais_nome, response.text)
        
        capital = soup.find('tr', id='places_capital__row')
        capital_nome = capital.find('td', class_='w2p_fw').text.strip() if capital else "Nao encontrado"

        currency = soup.find('tr', id='places_currency_name__row')
        currency_nome = currency.find('td', class_='w2p_fw').text.strip() if currency else "Nao encontrado"

        continent = soup.find('tr', id='places_continent__row')
        continent_nome = continent.find('td', class_='w2p_fw').text.strip() if continent else "Nao encontrado"

        return {
            'country': pais_nome,
            'capital': capital_nome,
            'currency': currency_nome,
            'continent': continent_nome,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    else:
        print(f"Erro ao acessar a pagina do pais {pais_link} (Status {response.status_code})")
        return None

In [4]:
def extrair_link_pais(soup):
    paises_links = []
    linhas = soup.select('table tr')
    
    for linha in linhas:
        colunas = linha.find_all('td')
        for coluna in colunas:
            link_tag = coluna.find('a')
            if link_tag:
                pais_nome = link_tag.text.strip()
                pais_link = url_pais + link_tag['href']
                paises_links.append((pais_nome, pais_link))
    return paises_links

In [5]:
def arq_csv(dados, nome_arquivo='dados_em_csv/dados_paises.csv'):
    arquivo_existe = os.path.isfile(nome_arquivo)

    with open(nome_arquivo, mode='a', newline='', encoding='utf-8') as arquivo:
        write = csv.DictWriter(arquivo, fieldnames=['country', 'capital', 'currency', 'continent', 'timestamp'])

        if not arquivo_existe:
            write.writeheader()

        write.writerow(dados)

In [6]:
def arq_html(nome_pais, conteudo_html):
    nome_arquivo = f"html_dos_sites/{nome_pais.replace(' ', '_')}.html"
    
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        arquivo.write(conteudo_html)
    print(f"A pagina HTML do pais {nome_pais} foi salva")

In [7]:
def crawl_all_pages(n_pgs=25):
    pg_atual = 0

    while pg_atual <= n_pgs:
        print(f"Processando a pagina {pg_atual}...")

        url = url_base + str(pg_atual)
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Erro ao acessar a pagina {pg_atual}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        paises_links = extrair_link_pais(soup)

        for pais_nome, pais_link in paises_links:
            pais_dados = extrair_dados_pais(pais_link)
            if pais_dados:
                arq_csv(pais_dados)

        pg_atual += 1
        time.sleep(1)

In [8]:
crawl_all_pages()

Processando a pagina 0...
A pagina HTML do pais Afghanistan foi salva
A pagina HTML do pais Aland Islands foi salva
A pagina HTML do pais Albania foi salva
A pagina HTML do pais Algeria foi salva
A pagina HTML do pais American Samoa foi salva
A pagina HTML do pais Andorra foi salva
A pagina HTML do pais Angola foi salva
A pagina HTML do pais Anguilla foi salva
A pagina HTML do pais Antarctica foi salva
A pagina HTML do pais Antigua and Barbuda foi salva
Processando a pagina 1...
A pagina HTML do pais Argentina foi salva
A pagina HTML do pais Armenia foi salva
A pagina HTML do pais Aruba foi salva
A pagina HTML do pais Australia foi salva
A pagina HTML do pais Austria foi salva
A pagina HTML do pais Azerbaijan foi salva
A pagina HTML do pais Bahamas foi salva
A pagina HTML do pais Bahrain foi salva
A pagina HTML do pais Bangladesh foi salva
A pagina HTML do pais Barbados foi salva
Processando a pagina 2...
A pagina HTML do pais Belarus foi salva
A pagina HTML do pais Belgium foi salva
A